# 수정예정

1. 시간을 현재시간 기준으로 조정
    1. 실패 : 1시간 부터 분단위가 없음
2. 이미 본 알바 제외시키기(시간을 기준으로)

In [0]:
from bs4 import BeautifulSoup
from tqdm import tqdm
from datetime import datetime
import requests
import pandas as pd


class Alba():
    def __init__(self):

        self.flag = False
        self.result_list = []
        self.words = ["물류", "상차", "하차", "상하차", "투잡", "배달", "식기", 
                     "세척", "발렛", "쿠팡", "남여", "드라마", "품평회", "허브",
                     "생동성", "임상시험", "전화", "서빙", "엑스트라", "영화", 
                     "조리보조", "주방", "남녀", "고3", "피킹", "Picking", 
                     "냉장", "오렌지라이프", "누구나", "택배", "분류",
                      "연회장", "판매"]
        self.data = pd.DataFrame(columns=["title", "area", "pay", "time", "recently", "link"])

    def crawler(self, number_pages):

        for page in tqdm(range(1, number_pages+1)):
            req = requests.get('http://www.albamon.com/list/gi/mon_gi_list.asp?page=' +str(page)+'&gubun=2&ps=50&ob=6&lvtype=1&rArea=,I000&sDutyTerm=,5,10,20&gender=M&gender_chk=1&rAge=31&rAge_Chk=1&rWDate=1&Empmnt_Type=&tc=529')
            req.encoding = None
            html = req.text
            soup = BeautifulSoup(html, 'html.parser')
            result_title = soup.select("#subcontent > form > div.gListWrap > table > tbody > tr > td.subject > div > p.cTit > a")
            result_area = soup.select("#subcontent > form > div.gListWrap > table > tbody > tr > td.area > div")
            result_pay = soup.select("#subcontent > form > div.gListWrap > table > tbody > tr > td.pay > p > img")
            result_recently = soup.select("#subcontent > form > div.gListWrap > table > tbody > tr > td.recently > em")

            for i in range(len(result_title)):
                self.data = self.data.append({"title" : result_title[i].text, 
                                            "area" : result_area[i].text, 
                                            "pay" : result_pay[i]['alt'],
                                            "time" : result_pay[i].findNext("td").text,
                                            "recently" : result_recently[i].text,
                                            "link" : "http://www.albamon.com/" + result_title[i]["href"]
                                            },
                                            ignore_index=True)
                    
        print("\n","crawling complete")


    def modifier(self):
        self.data["area"] = self.data["area"].apply(lambda x: x[6:-1] if True else x)
        self.data["time"] = self.data["time"].apply(lambda x: x[7:-7] if True else x)
        # self.data["recently"] = self.data["recently"].apply(lambda x: str(datetime.now().hour+9) + "시" + str(datetime.now().minute - int(x[:2])) + "분" if x[-2:] == "분전" and len(x) == 4 else x)
        # self.data["recently"] = self.data["recently"].apply(lambda x: str(datetime.now().hour+9) + "시" + str(datetime.now().minute - int(x[0])) + "분" if x[-2:] == "분전" and len(x) == 3 else x)
        # self.data["recently"] = self.data["recently"].apply(lambda x: str(datetime.now().hour+9 - int(x[0])) + "시" + str(datetime.now().minute)+ "분" if x[-2:] == "간전" else x)

        print("\n","modifier complete")

    def filter(self):
        drop_index_list = []
        for i in range(len(self.data)):
            for j in self.words:
                if j in self.data.title[i]:
                    drop_index_list.append(i)
                    continue
        self.data = self.data.drop(drop_index_list, axis=0)
        self.data = self.data.reset_index(drop=True)

        print("\n","filter complete")


In [0]:
alba = Alba()
alba.crawler(10)
alba.modifier()
alba.filter()
alba.data

 90%|█████████ | 9/10 [00:03<00:00,  2.32it/s]

In [0]:
alba.data.link[4]

'http://www.albamon.com//recruit/view/gi?AL_GI_No=67322679&mj_stat=0&optgf='

# test 코드

In [0]:
result_title[0]["href"]

'/recruit/view/gi?AL_GI_No=67262341&mj_stat=0&optgf='

In [0]:
result_area[0].text

'\n\n스크랩\n서울 중구 \n'

In [0]:
result_pay[0]['alt']

'일급'

In [0]:
result_pay[0].findNext("td").text

'\r\n\t\t\t\t\t시간협의\r\n\t\t\t\t\t'

In [0]:
result_recently[0]

<em>10분전</em>

In [0]:
len(result_title)

50

In [0]:
len(result_area)

50

In [0]:
len(result_pay)

50

In [0]:
len(result_recently)

50

In [0]:
test = pd.DataFrame(columns=["id", "title", "area", "pay", "time", "recently"])

test = test.append({"id" : 0, "title" : "a", "area" : "aa", "pay" : "aaa", "time" : "aaaa", "recently" : "aaaaa"}, ignore_index=True)
test

,id,title,area,pay,time,recently
0,0,a,aa,aaa,aaaa,aaaaa


In [0]:
alba.data.area[0][6:-1]

'서울 관악구'

In [0]:
alba.data.time[0][7:-7]

'15:30~00:30'

In [0]:
alba.data.recently[0]

<em>12분전</em>

In [0]:
datetime.now()

datetime.datetime(2019, 12, 17, 0, 17, 22, 854206)

In [0]:
print(datetime.now().hour + 9, datetime.now().minute)

9 20


In [0]:
len(alba.data.recently[0])

4

In [0]:
len(alba.data.recently[30])

4

In [0]:
alba.data.recently[0][-2:]

'분전'